In [ ]:
%matplotlib inline

Highly refernced from: 
https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials


DCGAN
=============================

#### 1. Importing Packages and Selecting a Device

In [ ]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

# Device configuration
device = 

#### 2. Hyper-parameters setting and load the images

In [ ]:
# Hyper-parameters
latent_size = 100
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'DCGAN'

In [ ]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
                                     std=(0.5, 0.5, 0.5))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='./MNIST/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

#### 3. Define Networks

In [ ]:
# Discriminator
D = nn.Sequential(
    nn.Conv2d(1, 128, 5, 1),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2),
    nn.Conv2d(128, 256, 5, 1),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2),
    nn.Conv2d(256, 512, 5, 1),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2),
    nn.Conv2d(512, 1024, 5, 2),
    nn.BatchNorm2d(1024),
    nn.LeakyReLU(0.2),
    nn.Conv2d(1024, 1, 5, 2),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.ConvTranspose2d(1, 1024, 3, 1),
    nn.BatchNorm2d(1024),
    nn.LeakyReLU(0.2),
    nn.ConvTranspose2d(1024, 512, 5, 1),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2),
    nn.ConvTranspose2d(512, 256, 5, 1),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2),
    nn.ConvTranspose2d(256, 128, 5, 1),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2),
    nn.ConvTranspose2d(128, 1, 5, 1),
    nn.Tanh())

In [ ]:
# Device setting
D = 
G = 

#### 4. Define loss function and optimizers

In [ ]:
# Binary cross entropy loss and optimizer
criterion = 
d_optimizer = 
g_optimizer = 

#### 5. Define denorm, reset_grad() functions (explained later)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

#### 6. Training

In [ ]:
# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = 
        fake_labels = 

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        
        # Compute BCELoss using real images
        #################################
        outputs = 
        d_loss_real = 
        real_score = outputs
        #################################
        
        
        # Compute BCELoss using fake images
        z = 
        
        # Reshape latent vector into (-1, 1, 10, 10) dimension
        z = 
        fake_images = 
        # Reshpae 1D vector into 3D tensor of (-1, 1, 28, 28)
        fake_images = 
        
        outputs = 
        d_loss_fake = 
        fake_score = outputs
        
        
        d_loss = 
        
        # Backprop and optimize Discriminators
        ###########################
        
        
        
        ###########################
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = 
        
        # Reshape latent vector into (-1, 1, 10, 10) dimension
        z = z.view(-1, 1, 10, 10)
        fake_images = 
        
        # Reshpae 1D vector into 3D tensor of (-1, 1, 28, 28)
        fake_images = 
        
        outputs = 
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        g_loss = 
        
        # Backprop and optimize
        ###########################
        
        
        
        ###########################
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                      real_score.mean().item(), fake_score.mean().item()))
        
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))